# 5.1 Model Reuse

In [19]:
import tensorflow as tf
import numpy as np

data=np.loadtxt('./data.csv', delimiter=',', unpack=True, dtype='float32') #unpack=True로 행과 열이 바뀜 
x_data=np.transpose(data[0:2])#load시 unpack=True로 행, 열이 반전되었기 때문에 다시 transpose 전치함
y_data=np.transpose(data[2:])

print('data\n',data)
print('x_data \n',x_data)
print('y_data \n',y_data)

"""
data2=np.loadtxt('./data.csv', delimiter=',',dtype='float32')
print('data2\n',data2)
"""

data
 [[0. 1. 1. 0. 0. 0.]
 [0. 0. 1. 0. 0. 1.]
 [1. 0. 0. 1. 1. 0.]
 [0. 1. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 1.]]
x_data 
 [[0. 0.]
 [1. 0.]
 [1. 1.]
 [0. 0.]
 [0. 0.]
 [0. 1.]]
y_data 
 [[1. 0. 0.]
 [0. 1. 0.]
 [0. 0. 1.]
 [1. 0. 0.]
 [1. 0. 0.]
 [0. 0. 1.]]


"\ndata2=np.loadtxt('./data.csv', delimiter=',',dtype='float32')\nprint('data2\n',data2)\n"

In [20]:
global_step=tf.Variable(0, trainable=False, name='global_step')

X=tf.placeholder(tf.float32)
Y=tf.placeholder(tf.float32)

W1=tf.Variable(tf.random_uniform([2,10],-1.,1.))
L1=tf.nn.relu(tf.matmul(X,W1))

W2=tf.Variable(tf.random_uniform([10,20],-1.,1.))
L2=tf.nn.relu(tf.matmul(L1,W2))

W3=tf.Variable(tf.random_uniform([20,3],-1.,1.))
model=tf.matmul(L2,W3)

cost=tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(labels=Y, logits=model))
optimizer=tf.train.AdamOptimizer(learning_rate=.01)
train_op=optimizer.minimize(cost,global_step=global_step)

In [21]:
#activate Session

sess=tf.Session()
saver=tf.train.Saver(tf.global_variables())

ckpt=tf.train.get_checkpoint_state('./model')
if ckpt and tf.train.checkpoint_exists(ckpt.model_checkpoint_path):
    saver.restore(sess, ckpt.model_checkpoint_path)
else:
    sess.run(tf.global_variables_initializer())

In [24]:
for step in range(2):
    sess.run(train_op, feed_dict={X:x_data, Y:y_data})
    
    print('Step:%d, ' %sess.run(global_step),
         'Cost:%.3f,' % sess.run(cost, feed_dict={X:x_data, Y:y_data}))
saver.save(sess, './model/dnn.ckpt', global_step=global_step)

Step:3,  Cost:1.163,
Step:4,  Cost:1.075,


'./model/dnn.ckpt-4'

In [23]:
prediction=tf.argmax(model,1)
target=tf.argmax(Y,1)
print('Prediction:',sess.run(prediction, feed_dict={X:x_data}))
print('Target:', sess.run(target, feed_dict={Y:y_data}))
is_correct=tf.equal(prediction,target)
accuracy=tf.reduce_mean(tf.cast(is_correct, tf.float32))
print('Accuracy:',sess.run(accuracy*100, feed_dict={X:x_data, Y:y_data}))

Prediction: [0 0 0 0 0 0]
Target: [0 1 2 0 0 2]
Accuracy: 50.0


# 5.2 Tensorboard

In [1]:
import tensorflow as tf
import numpy as np

data=np.loadtxt('./data.csv', delimiter=',', unpack=True, dtype='float32') #unpack=True로 행과 열이 바뀜 
x_data=np.transpose(data[0:2])#load시 unpack=True로 행, 열이 반전되었기 때문에 다시 transpose 전치함
y_data=np.transpose(data[2:])

global_step=tf.Variable(0, trainable=False, name='global_step')

X=tf.placeholder(tf.float32, name='X')
Y=tf.placeholder(tf.float32, name='Y')

with tf.name_scope('layer1'):
    W1=tf.Variable(tf.random_uniform([2,10],-1.,1.), name='W1')
    L1=tf.nn.relu(tf.matmul(X,W1))

with tf.name_scope('layer2'):
    W2=tf.Variable(tf.random_uniform([10,20],-1.,1.), name='W2')
    L2=tf.nn.relu(tf.matmul(L1,W2))

with tf.name_scope('output'):
    W3=tf.Variable(tf.random_uniform([20,3],-1.,1.), name='W3')
    model=tf.matmul(L2,W3)

with tf.name_scope('optimizer'):
    cost=tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(labels=Y, logits=model))
    optimizer=tf.train.AdamOptimizer(learning_rate=.01)
    train_op=optimizer.minimize(cost,global_step=global_step)
    tf.summary.scalar('cost',cost)

In [2]:
#activate Session

sess=tf.Session()
saver=tf.train.Saver(tf.global_variables())

ckpt=tf.train.get_checkpoint_state('./model')
if ckpt and tf.train.checkpoint_exists(ckpt.model_checkpoint_path):
    saver.restore(sess, ckpt.model_checkpoint_path)
else:
    sess.run(tf.global_variables_initializer())

merged=tf.summary.merge_all()
writer=tf.summary.FileWriter('./logs',sess.graph)

In [3]:
for step in range(100):
    sess.run(train_op, feed_dict={X:x_data, Y:y_data})
    
    print('Step:%d, ' %sess.run(global_step),
         'Cost:%.3f,' % sess.run(cost, feed_dict={X:x_data, Y:y_data}))
    summary=sess.run(merged, feed_dict={X:x_data, Y:y_data})
    writer.add_summary(summary, global_step=sess.run(global_step))
    
saver.save(sess, './model/dnn.ckpt', global_step=global_step)

prediction=tf.argmax(model,1)
target=tf.argmax(Y,1)
print('Prediction:',sess.run(prediction, feed_dict={X:x_data}))
print('Target:', sess.run(target, feed_dict={Y:y_data}))
is_correct=tf.equal(prediction,target)
accuracy=tf.reduce_mean(tf.cast(is_correct, tf.float32))
print('Accuracy:',sess.run(accuracy*100, feed_dict={X:x_data, Y:y_data}))

Step:1,  Cost:1.539,
Step:2,  Cost:1.461,
Step:3,  Cost:1.387,
Step:4,  Cost:1.318,
Step:5,  Cost:1.252,
Step:6,  Cost:1.190,
Step:7,  Cost:1.132,
Step:8,  Cost:1.077,
Step:9,  Cost:1.025,
Step:10,  Cost:0.981,
Step:11,  Cost:0.940,
Step:12,  Cost:0.901,
Step:13,  Cost:0.865,
Step:14,  Cost:0.831,
Step:15,  Cost:0.801,
Step:16,  Cost:0.773,
Step:17,  Cost:0.749,
Step:18,  Cost:0.727,
Step:19,  Cost:0.709,
Step:20,  Cost:0.693,
Step:21,  Cost:0.678,
Step:22,  Cost:0.665,
Step:23,  Cost:0.653,
Step:24,  Cost:0.643,
Step:25,  Cost:0.633,
Step:26,  Cost:0.624,
Step:27,  Cost:0.616,
Step:28,  Cost:0.609,
Step:29,  Cost:0.602,
Step:30,  Cost:0.596,
Step:31,  Cost:0.591,
Step:32,  Cost:0.587,
Step:33,  Cost:0.583,
Step:34,  Cost:0.579,
Step:35,  Cost:0.576,
Step:36,  Cost:0.573,
Step:37,  Cost:0.571,
Step:38,  Cost:0.569,
Step:39,  Cost:0.567,
Step:40,  Cost:0.566,
Step:41,  Cost:0.564,
Step:42,  Cost:0.563,
Step:43,  Cost:0.562,
Step:44,  Cost:0.561,
Step:45,  Cost:0.560,
Step:46,  Cost:0.55